# Overlaying the segmented images on the truecolor images


In [29]:
import os
import pandas as pd
import skimage
from skimage.color import label2rgb
import numpy as np
import rasterio as rio
from rasterio.plot import reshape_as_image
from skimage.transform import resize
import proplot as pplt
regions = pd.read_csv('../data/metadata/region_definitions.csv', index_col='region')
cases = pd.read_csv('../data/metadata/validation_dataset_case_list.csv')
cases['start_date'] = pd.to_datetime(cases['start_date'].values)
saveloc = '../data/modis/'
ebseg_loc = '../data/ift_images/ebseg_v0/'
labeled_loc = '../data/validation_images/'

In [32]:
for row, case in cases.iterrows():
    cn = str(case.case_number).zfill(3)
    region = case.region
    start = case.start_date # check start date format
    end = case.start_date + pd.to_timedelta('1d')
    dx = '100km'

    
    for satellite in ['aqua', 'terra']:
        case_folder = '-'.join([cn, region, dx, start.strftime('%Y%m%d'), end.strftime('%Y%m%d')])
        case_folder += '/' + '-'.join([region, start.strftime('%Y-%m-%d'), satellite])
        ebseg_path = ebseg_loc +  '-'.join(
            [cn, region, dx, start.strftime('%Y%m%d'), satellite, 'ebseg_v0']) + '.tiff'
    
        if os.path.isfile(ebseg_path):

            # Load truecolor image
            with rio.open(
                saveloc + 'truecolor' + '/' + '.'.join(['-'.join([cn, region, dx, start.strftime('%Y%m%d')]),
                                                       satellite, 'truecolor', '250m', 'tiff'])) as truecolor_image:
                tc_imdata = truecolor_image.read()
                tc_bounds = truecolor_image.bounds

            # Load labeled image
            with rio.open(ebseg_loc + '-'.join(
                [cn, region, dx, start.strftime('%Y%m%d'), satellite, 'ebseg_v0']) + '.tiff') as ebseg_labeled_image:
                # ebseg_imdata = resize(ebseg_labeled_image.read().squeeze(), (391, 391), order=0)
                ebseg_imdata = ebseg_labeled_image.read().squeeze()
            left, bottom, right, top = tc_bounds

            # Overlay colored labels on the TC image
            fig, ax = pplt.subplots(width=6)
            if np.any(ebseg_imdata > 0):
                overlay_ebseg = label2rgb(label=ebseg_imdata,
                                           image=reshape_as_image(tc_imdata),
                                           kind='overlay', alpha=0.5,
                                           bg_color=None,
                                           bg_label=0)
                ax.imshow(overlay_ebseg, extent=[left, right, top, bottom])

            else:
                ax.imshow(reshape_as_image(tc_imdata), extent=[left, right, top, bottom])

            # If manually labeled floes exist, overlay an outline
            label_path = labeled_loc + 'labeled_floes_png/' + \
                                           '-'.join([cn, region, start.strftime('%Y%m%d'),
                                                     satellite, 'labeled_floes']) + '.png'
            if os.path.isfile(label_path):
                manually_labeled_image = skimage.io.imread(label_path)[:,:,0]
                eroded_image = skimage.morphology.erosion(manually_labeled_image, skimage.morphology.disk(1))
                borders = np.ma.masked_array(manually_labeled_image, mask=manually_labeled_image - eroded_image == 0) 
                ax.imshow(borders, color='k', alpha=1, extent=[left, right, top, bottom])
                
            ax.axis('off')
            fig.save('../data/ift_images/overlays/' + \
                     '-'.join([cn, region, dx, start.strftime('%Y%m%d'), satellite, 'ebseg_v0']) + '.png',
                     dpi=300, bbox_inches='tight', pad_inches=0)
            pplt.close(fig)
        else:
            print(case.case_number)
       


0


C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\cor

82
82
89


C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\cor

144


...

363
372


C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)


378
378


C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)


386
386
387
466
401
457
457
464
464
461
431
431
440
440


In [31]:
ebseg_imdata.shape

(1, 391, 391)

In [89]:
left, bottom, right, top = ebseg_labeled_image.bounds

In [90]:
(right - left)/400

250.0